In [1]:
from tinyagents import chainable, respond

@chainable
class Agent:
    name: str = "main_agent"

    def __init__(self):
        pass

    def run(self, input: str):
        return "Hello! I was provided with some tool outputs: " + str(input)
    
    def output_handler(self, output):
        return respond(output)

@chainable
class Tool1:
    """ Some tool """
    name: str = "tool1"

    def run(self, input):
        return "Tool 1 has been triggered."

@chainable
class Tool2:
    """ Some other tool """
    def __init__(self, phrase: str):
        self.phrase = phrase

    def run(self, input):
        return f"Tool 2 has been {self.phrase}."

2024-07-15 21:32:08,876	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
branch1 = Tool1() / Tool2(phrase="executed")

def router(x):
    if x.startswith("blue"):
        return Tool1.name
    
    return Tool2.name
    

branch1.bind_router(router)

ConditionalBranch(tool1 | Tool2)

In [3]:
graph = branch1 | Agent()
str(graph)

'ConditionalBranch(tool1 | Tool2) -> main_agent'

In [4]:
# Run without using Ray
runner = graph.compile()
runner.invoke("blue!")


 > Running node: ConditionalBranch(tool1 | Tool2)

	Input: blue!

	Output (ConditionalBranch(tool1 | Tool2)): {
  "content": "Tool 1 has been triggered.",
  "action": null,
  "ref": null
}

 > Running node: main_agent

	Input: Tool 1 has been triggered.

	Output (main_agent): {
  "content": "Hello! I was provided with some tool outputs: Tool 1 has been triggered.",
  "action": "respond",
  "ref": null
}


'Hello! I was provided with some tool outputs: Tool 1 has been triggered.'

In [5]:
runner.invoke("yellow!")


 > Running node: ConditionalBranch(tool1 | Tool2)

	Input: yellow!

	Output (ConditionalBranch(tool1 | Tool2)): {
  "content": "Tool 2 has been executed.",
  "action": null,
  "ref": null
}

 > Running node: main_agent

	Input: Tool 2 has been executed.

	Output (main_agent): {
  "content": "Hello! I was provided with some tool outputs: Tool 2 has been executed.",
  "action": "respond",
  "ref": null
}


'Hello! I was provided with some tool outputs: Tool 2 has been executed.'

In [6]:
# Run using Ray
runner = graph.compile(
    use_ray=True,
    ray_options={
        "tool1": {
            "num_replicas": 3
        },
        "tool2": {
            "num_replicas": 3,
            # "placement_group_strategy": ... 
        },
        "runner": {
            "num_replicas": 2
        }
    }
)
runner

In [7]:
from ray import serve

# we can now serve it using Ray Serve

app = serve.run(runner, name="MyApp")

2024-07-15 21:32:10,969	INFO worker.py:1762 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
(ProxyActor pid=64742) INFO 2024-07-15 21:32:12,393 proxy 127.0.0.1 proxy.py:1179 - Proxy starting on node 0fd9c24479671e71cf618179e405fb840349464b2039ce7bc00a1e98 (HTTP port: 8000).2024-07-15 21:32:12,427	INFO handle.py:126 -- Created DeploymentHandle '1frzrxl6' for Deployment(name='tool1', app='MyApp').

2024-07-15 21:32:12,428	INFO handle.py:126 -- Created DeploymentHandle 'ip857yee' for Deployment(name='Tool2', app='MyApp').
2024-07-15 21:32:12,429	INFO handle.py:126 -- Created DeploymentHandle 'zi2k7jw0' for Deployment(name='main_agent', app='MyApp').
2024-07-15 21:32:12,429	INFO handle.py:126 -- Created DeploymentHandle 'qr7ihs9p' for Deployment(name='tool1', app='MyApp').
2024-07-15 21:32:12,429	INFO handle.py:126 -- Created DeploymentHandle 'jsfg9ulz' for Deployment(name='Tool2', app='MyApp').
2024-07-15 21:32:12,429	INFO handle.py:126 -- Created DeploymentH

In [8]:
!serve status

proxies:
  0fd9c24479671e71cf618179e405fb840349464b2039ce7bc00a1e98: HEALTHY
applications:
  MyApp:
    status: RUNNING
    message: ''
    last_deployed_time_s: 1721075532.433455
    deployments:
      tool1:
        status: HEALTHY
        status_trigger: CONFIG_UPDATE_COMPLETED
        replica_states:
          RUNNING: 3
        message: ''
      Tool2:
        status: HEALTHY
        status_trigger: CONFIG_UPDATE_COMPLETED
        replica_states:
          RUNNING: 1
        message: ''
      main_agent:
        status: HEALTHY
        status_trigger: CONFIG_UPDATE_COMPLETED
        replica_states:
          RUNNING: 1
        message: ''
      runner:
        status: HEALTHY
        status_trigger: CONFIG_UPDATE_COMPLETED
        replica_states:
          RUNNING: 2
        message: ''
target_capacity: null


In [9]:
result = await app.ainvoke.remote("blue!")
result

2024-07-15 21:32:15,411	INFO handle.py:126 -- Created DeploymentHandle 's0k9do12' for Deployment(name='runner', app='MyApp').
2024-07-15 21:32:15,416	INFO pow_2_scheduler.py:260 -- Got updated replicas for Deployment(name='runner', app='MyApp'): {'y2kvw95e', 'loqbid8f'}.


NodeOutput(content='Hello! I was provided with some tool outputs: Tool 1 has been triggered.', action=<Action.Respond: 'respond'>, ref=None)

In [11]:
# we can also interact with the application via REST

import requests

response = requests.post("http://localhost:8000/invoke", data="blue")

print(response.text)

{"content":"Hello! I was provided with some tool outputs: Tool 1 has been triggered.","action":"respond","ref":null}
